In [2]:
!pip install emoji
!pip install xlrd
!pip install openpyxl

  Using cached emoji-2.14.1-py3-none-any.whl.metadata (5.7 kB)
Using cached emoji-2.14.1-py3-none-any.whl (590 kB)
  Using cached xlrd-2.0.1-py2.py3-none-any.whl.metadata (3.4 kB)
Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)


In [5]:
import re
import unicodedata
import emoji
import pandas as pd 

# 반복 문자 축약 대상 (초성, 자음, 특수문자)
REPEAT_CHAR_PATTERN = re.compile(r'([ㅋㅎㅠㅜㅇㄱ.,!?])\1{1,}', re.UNICODE)

# URL, 이메일, mojibake(�), 공백 패턴
url_pattern = re.compile(r'https?:\/\/[^\s]+')
email_pattern = re.compile(r'\S+@\S+')
mojibake_pattern = re.compile(r'�+')

# 이모지 제거 함수
def remove_emojis(text):
    return emoji.replace_emoji(text, replace='')

# 반복 문자 정규화 함수
def normalize_repeats(text, repeat_limit=2):
    return REPEAT_CHAR_PATTERN.sub(lambda m: m.group(1) * repeat_limit, text)

# 최종 클렌징 함수
def clean_text(text: str, keep_english: bool = True) -> str:
    if not isinstance(text, str):  # 예외처리: 결측치 등 비문자 처리
        return ""

    # 1. 유니코드 정규화
    text = unicodedata.normalize("NFKC", text)

    # 2. URL, 이메일 제거
    text = url_pattern.sub('', text)
    text = email_pattern.sub('', text)

    # 3. mojibake 문자 제거
    text = mojibake_pattern.sub('', text)

    # 4. 반복 문자 정규화 (ㅋㅋㅋ → ㅋㅋ, ㅠㅠㅠㅠ → ㅠㅠ 등)
    text = normalize_repeats(text, repeat_limit=2)

    # 5. 이모지 제거
    text = remove_emojis(text)

    # 6. 특수문자 정리
    if keep_english:
        pattern = re.compile(r"[^ .,?!/@$%~％·∼()\x00-\x7Fㅏ-ㅣㄱ-ㅎ가-힣]+")
    else:
        pattern = re.compile(r"[^ .,?!/@$%~％·∼()ㅏ-ㅣㄱ-ㅎ가-힣]+")

    # 7. 공백 정리
    text = re.sub(r"\s+", " ", text).strip()

    return text

# 엑셀 클렌징 실행
file_path = "에이블리_데이터선정_조끼_테스트용.xlsx"
df = pd.read_excel(file_path, engine='openpyxl')

df["클렌징"] = df["원문"].astype(str).apply(clean_text)

# 같은 파일에 덮어쓰기
df.to_excel(file_path, index=False)